In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle as pkl
import glob
import numpy as np
import cv2
from img_load import load_bimg
from NPSmethods import *

In [ ]:
psat=277

#Settings of imaging system
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

#Configurations for NPSMethods (here we only need do_Fit)
choices = {
    "if_Save"                     : False,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : False,
    "plot_NoisePowSpec_LineCut"   : False,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

#Directory containing image folders
path = r'C:\directory\raw_image'

#Directory where ML data will output to
resDir = r"C:\directory\results"

trapRegion = (slice(100, 200), slice(100, 200))
    # the region where the atoms located, [(xmin, xmax), (ymin, ymax)] (pixel)

noiseRegion = (slice(0, 300), slice(0, 300))
    # the region chosen for noise analysis, [(xmin, xmax), (ymin, ymax)] (pixel)

numOfImgsInEachRun = 50      # number of images for each run of analysis

rampingParameter = 0

#Lists where OD images and parameters will be stored
rawimages = []
parameters = []

#Sort through folders in path directory for raw images, extract OD image and parameters, then append to lists
for folder in glob.iglob(path + "\\*"):
    result=load_bimg(folder + "\\", raw=True)
    odimg=-np.log(result['raw1']/result['raw2'])-(result['raw1']-result['raw2'])/psat
    _, _, _, _, popt, _ = doCalibration(folder, resDir, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    for i in range(len(odimg)):
        rawimages.append(odimg[i])
        parameters.append(popt)


In [ ]:
#Scale data and split into training and validation sets as arrays
scaler = StandardScaler()
yscaler = scaler.fit(np.array(parameters))
scaledpara = scaler.transform(np.array(parameters))
rawimages = np.array(rawimages).reshape((147,-1))
trainin, testin, trainout, testout = train_test_split(rawimages, scaledpara, test_size=0.25, shuffle=True)

In [ ]:
#Store ML training data as pickle files
pkl.dump(trainin, open(resDir + "\\trainin.pkl", "wb"))
pkl.dump(trainout, open(resDir + "\\trainout.pkl", "wb"))
pkl.dump(testin, open(resDir + "\\testin.pkl", "wb"))
pkl.dump(testout, open(resDir + "\\testout.pkl", "wb"))
pkl.dump(yscaler, open(resDir + "\\yscaler.pkl", "wb"))